<h1 style="color:DodgerBlue">Индивидальный проект</h1>

<h2 style="color:DodgerBlue">Название проекта:</h2>

----

### Вариант задания 


<h2 style="color:DodgerBlue">Описание проекта:</h2>

----

[ваш текст]

#### Дополнительное задание
Добавьте к сущестующим классам (базовыму и производным 3-4 атрибута и метода) создайте явную реализации интерфейса и управление зависимостями 


<h2 style="color:DodgerBlue">Реализация:</h2>

----

In [4]:
using System;
using System.Collections.Generic;

// Интерфейс 1: Система уведомлений
public interface INotificationService
{
    void SendNotification(string message, string studentId);
}

// Интерфейс 2: Система журналирования событий
public interface IEventLogger
{
    void LogEvent(string studentId, string eventType, string details, DateTime timestamp);
}

// Интерфейс 3: Оценки и успеваемость
public interface IGradeable
{
    void AddGrade(string subject, decimal grade);
    decimal CalculateAverageGrade();
    List<string> GetSubjects();
}

// Интерфейс 4: Исследовательская деятельность
public interface IResearcher
{
    void AddPublication(string title);
    int GetPublicationCount();
    List<string> GetPublications();
}

// Интерфейс 5: Преподавательская деятельность
public interface ITeacherAssistant
{
    void ConductSeminar(string subject);
    void GradeAssignments(int count);
}

// Реализация Email уведомлений
public class EmailNotificationService : INotificationService
{
    public void SendNotification(string message, string studentId)
    {
        Console.WriteLine($"Уведомление студенту {studentId}: {message}");
    }
}

// Реализация журналирования в консоль
public class ConsoleEventLogger : IEventLogger
{
    public void LogEvent(string studentId, string eventType, string details, DateTime timestamp)
    {
        Console.WriteLine($"Журнал: Студент {studentId}, Событие: {eventType}, Детали: {details}, Время: {timestamp:HH:mm:ss}");
    }
}

// Базовый класс Person с зависимостями
public class Person
{
    public string Name { get; protected set; }
    public int Age { get; protected set; }
    public string Email { get; protected set; }
    public string Phone { get; protected set; }
    public DateTime RegistrationDate { get; protected set; }
    public bool IsActive { get; protected set; }
    
    public string Address { get; protected set; }
    public string PassportNumber { get; protected set; }
    public DateTime LastActivityDate { get; protected set; }
    public int ActivityCount { get; protected set; }
    
    protected readonly INotificationService _notificationService;
    protected readonly IEventLogger _eventLogger;

    public Person(string name, int age, string email, string phone, string address, string passportNumber,
                  INotificationService notificationService, IEventLogger eventLogger)
    {
        Name = name;
        Age = age;
        Email = email;
        Phone = phone;
        Address = address;
        PassportNumber = passportNumber;
        RegistrationDate = DateTime.Now;
        IsActive = true;
        LastActivityDate = DateTime.Now;
        ActivityCount = 0;
        
        _notificationService = notificationService;
        _eventLogger = eventLogger;
        
        _eventLogger.LogEvent("NEW_PERSON", "PERSON_CREATED", $"Создана персона: {name}, возраст: {age}", DateTime.Now);
    }

    protected virtual void UpdateActivity()
    {
        LastActivityDate = DateTime.Now;
        ActivityCount++;
    }

    public virtual void ChangeContactInfo(string newEmail, string newPhone)
    {
        string oldEmail = Email;
        string oldPhone = Phone;
        
        Email = newEmail;
        Phone = newPhone;
        
        UpdateActivity();
        _eventLogger.LogEvent("PERSON_" + Name.Replace(" ", "_"), "CONTACT_CHANGED", 
                              $"Изменены контакты: email {oldEmail}->{newEmail}, phone {oldPhone}->{newPhone}", DateTime.Now);
        _notificationService.SendNotification($"Контактная информация обновлена: email={newEmail}, phone={newPhone}", "PERSON_" + Name.Replace(" ", "_"));
        
        Console.WriteLine($"Контактная информация обновлена для {Name}: email={newEmail}, phone={newPhone}");
    }

    public virtual void Suspend()
    {
        IsActive = false;
        UpdateActivity();
        _eventLogger.LogEvent("PERSON_" + Name.Replace(" ", "_"), "SUSPENDED", "Персона приостановлена", DateTime.Now);
        _notificationService.SendNotification("Ваш статус приостановлен", "PERSON_" + Name.Replace(" ", "_"));
        
        Console.WriteLine($"{Name} приостановлен");
    }

    public virtual void Activate()
    {
        IsActive = true;
        UpdateActivity();
        _eventLogger.LogEvent("PERSON_" + Name.Replace(" ", "_"), "ACTIVATED", "Персона активирована", DateTime.Now);
        _notificationService.SendNotification("Ваш статус активирован", "PERSON_" + Name.Replace(" ", "_"));
        
        Console.WriteLine($"{Name} активирован");
    }

    public virtual void DisplayInfo()
    {
        Console.WriteLine($"Персона: {Name}, Возраст: {Age}, Email: {Email}, Телефон: {Phone}");
    }

    public virtual void DisplayInfo(bool detailed)
    {
        DisplayInfo();
        if (detailed)
        {
            Console.WriteLine($"Адрес: {Address}, Паспорт: {PassportNumber}");
            Console.WriteLine($"Дата регистрации: {RegistrationDate:dd.MM.yyyy}, Статус: {(IsActive ? "Активен" : "Неактивен")}");
            Console.WriteLine($"Активность: операций {ActivityCount}, Последняя: {LastActivityDate:HH:mm}");
            Console.WriteLine($"Дней с регистрации: {(DateTime.Now - RegistrationDate).Days}");
        }
    }
}

// Базовый класс Student
public class Student : Person, IGradeable
{
    public string StudentId { get; protected set; }
    public int Course { get; protected set; }
    public decimal AverageGrade { get; protected set; }
    public bool IsOnScholarship { get; protected set; }
    public List<string> EnrolledSubjects { get; protected set; }
    public int AcademicHours { get; protected set; } // Академические часы
    
    private Dictionary<string, List<decimal>> grades;

    public Student(string name, int age, string email, string phone, string address, string passportNumber,
                   string studentId, int course, decimal averageGrade, bool isOnScholarship,
                   INotificationService notificationService, IEventLogger eventLogger)
        : base(name, age, email, phone, address, passportNumber, notificationService, eventLogger)
    {
        StudentId = studentId;
        Course = course;
        AverageGrade = averageGrade;
        IsOnScholarship = isOnScholarship;
        EnrolledSubjects = new List<string>();
        AcademicHours = 0;
        grades = new Dictionary<string, List<decimal>>();
        
        _eventLogger.LogEvent(studentId, "STUDENT_CREATED", $"Создан студент: {name}, курс: {course}, средний балл: {averageGrade}", DateTime.Now);
    }

    public override void DisplayInfo()
    {
        Console.WriteLine($"Студент: {Name}, ID: {StudentId}, Возраст: {Age}, Курс: {Course}, Средний балл: {AverageGrade:F2}, Стипендия: {(IsOnScholarship ? "Да" : "Нет")}");
    }

    public virtual void Study()
    {
        Console.WriteLine($"{Name} учится на {Course} курсе.");
        AcademicHours += 36; // Стандартные академические часы за неделю
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "STUDY", $"Изучение материалов на курсе {Course}", DateTime.Now);
    }

    public virtual void Study(string subject)
    {
        Console.WriteLine($"{Name} изучает предмет '{subject}' на {Course} курсе.");
        if (!EnrolledSubjects.Contains(subject))
        {
            EnrolledSubjects.Add(subject);
        }
        AcademicHours += 18; // Часы по предмету
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "STUDY_SUBJECT", $"Изучение предмета: {subject}", DateTime.Now);
    }

    public virtual void TakeExam()
    {
        Console.WriteLine($"{Name} сдаёт экзамен на {Course} курсе.");
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "EXAM", "Сдача экзамена", DateTime.Now);
    }

    public virtual void TakeExam(string subject)
    {
        Console.WriteLine($"{Name} сдаёт экзамен по предмету '{subject}' на {Course} курсе.");
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "EXAM_SUBJECT", $"Сдача экзамена по предмету: {subject}", DateTime.Now);
    }

    // Реализация методов IGradeable
    void IGradeable.AddGrade(string subject, decimal grade)
    {
        if (!grades.ContainsKey(subject))
        {
            grades[subject] = new List<decimal>();
        }
        grades[subject].Add(grade);
        
        // Пересчет среднего балла
        decimal total = 0;
        int count = 0;
        foreach (var subjectGrades in grades.Values)
        {
            foreach (var g in subjectGrades)
            {
                total += g;
                count++;
            }
        }
        AverageGrade = count > 0 ? total / count : 0;
        
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "GRADE_ADDED", $"Добавлена оценка: {subject} - {grade:F2}", DateTime.Now);
        _notificationService.SendNotification($"Добавлена оценка по предмету {subject}: {grade:F2}", StudentId);
        
        Console.WriteLine($"Добавлена оценка для {Name} по предмету {subject}: {grade:F2}");
    }

    decimal IGradeable.CalculateAverageGrade()
    {
        return AverageGrade;
    }

    List<string> IGradeable.GetSubjects()
    {
        return new List<string>(grades.Keys);
    }

    public override void Suspend()
    {
        base.Suspend();
        _eventLogger.LogEvent(StudentId, "STUDENT_SUSPENDED", "Студент отстранен от занятий", DateTime.Now);
        Console.WriteLine($"Студент {Name} отстранен от занятий");
    }

    public override void Activate()
    {
        base.Activate();
        _eventLogger.LogEvent(StudentId, "STUDENT_ACTIVATED", "Студент восстановлен", DateTime.Now);
        Console.WriteLine($"Студент {Name} восстановлен");
    }

    public void EnrollSubject(string subject)
    {
        if (!EnrolledSubjects.Contains(subject))
        {
            EnrolledSubjects.Add(subject);
            UpdateActivity();
            _eventLogger.LogEvent(StudentId, "SUBJECT_ENROLLED", $"Запись на предмет: {subject}", DateTime.Now);
            _notificationService.SendNotification($"Вы записаны на предмет: {subject}", StudentId);
            
            Console.WriteLine($"{Name} записался на предмет: {subject}");
        }
    }

    public virtual void AdvanceToNextCourse()
    {
        Course++;
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "COURSE_ADVANCE", $"Переход на курс {Course}", DateTime.Now);
        _notificationService.SendNotification($"Вы переведены на {Course} курс", StudentId);
        
        Console.WriteLine($"{Name} переведен на {Course} курс");
    }

    public virtual decimal CalculateScholarship()
    {
        if (IsOnScholarship && AverageGrade >= 4.0m)
        {
            decimal scholarship = AverageGrade * 1000;
            _eventLogger.LogEvent(StudentId, "SCHOLARSHIP_CALCULATED", $"Рассчитана стипендия: {scholarship:C}", DateTime.Now);
            return scholarship;
        }
        return 0;
    }
}

// Класс BachelorStudent
public class BachelorStudent : Student
{
    public string Specialty { get; private set; }
    public string Faculty { get; private set; }
    public bool HasInternship { get; private set; }
    public string StudyForm { get; private set; }
    public int ExamsPassed { get; private set; }
    
    public BachelorStudent(string name, int age, string email, string phone, string address, string passportNumber,
                          string studentId, int course, decimal averageGrade, bool isOnScholarship,
                          string specialty, string faculty, bool hasInternship, string studyForm,
                          INotificationService notificationService, IEventLogger eventLogger)
        : base(name, age, email, phone, address, passportNumber, studentId, course, averageGrade, isOnScholarship, 
              notificationService, eventLogger)
    {
        Specialty = specialty;
        Faculty = faculty;
        HasInternship = hasInternship;
        StudyForm = studyForm;
        ExamsPassed = 0;
        
        _eventLogger.LogEvent(studentId, "BACHELOR_CREATED", $"Создан бакалавр: {name}, специальность: {specialty}, факультет: {faculty}", DateTime.Now);
    }

    public override void DisplayInfo()
    {
        Console.WriteLine($"Бакалавр: {Name}, Специальность: {Specialty}, Факультет: {Faculty}, Курс: {Course}, Форма: {StudyForm}, Стажировка: {(HasInternship ? "Да" : "Нет")}, Сдано экзаменов: {ExamsPassed}");
    }

    public override void Study()
    {
        Console.WriteLine($"{Name} (бакалавр) изучает предметы по специальности '{Specialty}' на факультете {Faculty}.");
        AcademicHours += 54; // У бакалавров больше часов
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "BACHELOR_STUDY", $"Изучение специальности: {Specialty}", DateTime.Now);
    }

    public void CompleteInternship()
    {
        HasInternship = true;
        ExamsPassed += 5; // За стажировку засчитываются экзамены
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "INTERNSHIP_COMPLETED", $"Завершена стажировка по специальности: {Specialty}", DateTime.Now);
        _notificationService.SendNotification($"Вы завершили стажировку по специальности {Specialty}", StudentId);
        
        Console.WriteLine($"{Name} завершил стажировку по специальности {Specialty}");
    }

    public override void AdvanceToNextCourse()
    {
        base.AdvanceToNextCourse();
        ExamsPassed += 8; // За курс сдается примерно 8 экзаменов
        _eventLogger.LogEvent(StudentId, "BACHELOR_ADVANCE", $"Бакалавр переведен на курс {Course}, сдано экзаменов: {ExamsPassed}", DateTime.Now);
        
        Console.WriteLine($"{Name} сдал 8 экзаменов. Всего сдано экзаменов: {ExamsPassed}");
    }

    public bool IsEligibleForGraduation()
    {
        // В российской системе: 4 курса бакалавриата, стажировка
        bool eligible = Course >= 4 && ExamsPassed >= 32 && HasInternship;
        _eventLogger.LogEvent(StudentId, "GRADUATION_CHECK", $"Проверка на выпуск: курс={Course}, экзамены={ExamsPassed}, стажировка={HasInternship}, результат={eligible}", DateTime.Now);
        return eligible;
    }
}

// Класс MasterStudent с интерфейсами
public class MasterStudent : Student, IResearcher
{
    public string ScientificAdvisor { get; private set; }
    public string ResearchTopic { get; private set; }
    public bool HasPublications { get; private set; }
    private int _publicationsCount;
    private List<string> _publications;
    public int ResearchHours { get; private set; }

    public MasterStudent(string name, int age, string email, string phone, string address, string passportNumber,
                        string studentId, int course, decimal averageGrade, bool isOnScholarship,
                        string scientificAdvisor, string researchTopic, bool hasPublications,
                        INotificationService notificationService, IEventLogger eventLogger)
        : base(name, age, email, phone, address, passportNumber, studentId, course, averageGrade, isOnScholarship, 
              notificationService, eventLogger)
    {
        ScientificAdvisor = scientificAdvisor;
        ResearchTopic = researchTopic;
        HasPublications = hasPublications;
        _publicationsCount = 0;
        _publications = new List<string>();
        ResearchHours = 0;
        
        _eventLogger.LogEvent(studentId, "MASTER_CREATED", $"Создан магистрант: {name}, научный руководитель: {scientificAdvisor}, тема: {researchTopic}", DateTime.Now);
    }

    public override void DisplayInfo()
    {
        Console.WriteLine($"Магистрант: {Name}, Научный руководитель: {ScientificAdvisor}, Тема: {ResearchTopic}, Публикаций: {_publicationsCount}, Курс: {Course}, Исследовательских часов: {ResearchHours}");
    }

    public override void TakeExam()
    {
        Console.WriteLine($"{Name} (магистрант) сдаёт экзамен под руководством {ScientificAdvisor}. Тема исследования: {ResearchTopic}");
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "MASTER_EXAM", $"Сдача экзамена под руководством {ScientificAdvisor}", DateTime.Now);
    }

    // Явная реализация методов IResearcher
    void IResearcher.AddPublication(string title)
    {
        _publicationsCount++;
        HasPublications = true;
        _publications.Add(title);
        
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "PUBLICATION_ADDED", $"Добавлена публикация: {title}", DateTime.Now);
        _notificationService.SendNotification($"Ваша статья '{title}' опубликована", StudentId);
        
        Console.WriteLine($"{Name} опубликовал статью: '{title}'");
        Console.WriteLine($"Всего публикаций: {_publicationsCount}");
    }

    int IResearcher.GetPublicationCount()
    {
        return _publicationsCount;
    }

    List<string> IResearcher.GetPublications()
    {
        return new List<string>(_publications);
    }

    public void ConsultWithAdvisor()
    {
        Console.WriteLine($"{Name} консультируется с научным руководителем {ScientificAdvisor} по теме: {ResearchTopic}");
        ResearchHours += 2;
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "ADVISOR_CONSULTATION", $"Консультация с {ScientificAdvisor}", DateTime.Now);
    }

    public override void AdvanceToNextCourse()
    {
        base.AdvanceToNextCourse();
        _eventLogger.LogEvent(StudentId, "MASTER_ADVANCE", $"Магистрант переведен на курс {Course}", DateTime.Now);
        
        if (_publicationsCount < 2)
        {
            Console.WriteLine($"Рекомендуется увеличить количество публикаций. Текущее: {_publicationsCount}");
        }
    }
}

// Класс PhDStudent с множественными интерфейсами
public class PhDStudent : Student, IResearcher, ITeacherAssistant
{
    public string DissertationTopic { get; private set; }
    public string Department { get; private set; }
    public DateTime DefenseDate { get; private set; }
    public bool IsTeaching { get; private set; }
    public string ResearchGroup { get; private set; }
    
    private int _publicationsCount;
    private List<string> _publications;
    private int _taughtSeminars;
    public int TeachingHours { get; private set; }

    public PhDStudent(string name, int age, string email, string phone, string address, string passportNumber,
                     string studentId, int course, decimal averageGrade, bool isOnScholarship,
                     string dissertationTopic, string department, DateTime defenseDate, bool isTeaching, string researchGroup,
                     INotificationService notificationService, IEventLogger eventLogger)
        : base(name, age, email, phone, address, passportNumber, studentId, course, averageGrade, isOnScholarship, 
              notificationService, eventLogger)
    {
        DissertationTopic = dissertationTopic;
        Department = department;
        DefenseDate = defenseDate;
        IsTeaching = isTeaching;
        ResearchGroup = researchGroup;
        _publicationsCount = 0;
        _publications = new List<string>();
        _taughtSeminars = 0;
        TeachingHours = 0;
        
        _eventLogger.LogEvent(studentId, "PHD_CREATED", $"Создан аспирант: {name}, тема: {dissertationTopic}, кафедра: {department}, защита: {defenseDate:dd.MM.yyyy}", DateTime.Now);
    }

    public override void DisplayInfo()
    {
        int daysLeft = (DefenseDate - DateTime.Now).Days;
        Console.WriteLine($"Аспирант: {Name}, Тема: '{DissertationTopic}', Кафедра: {Department}, Группа: {ResearchGroup}, Дней до защиты: {daysLeft}, Преподавание: {(IsTeaching ? "Да" : "Нет")}, Публикаций: {_publicationsCount}, Проведено семинаров: {_taughtSeminars}, Преподавательских часов: {TeachingHours}");
    }

    public override void Study()
    {
        Console.WriteLine($"{Name} (аспирант) проводит исследование по теме '{DissertationTopic}' в группе {ResearchGroup}.");
        AcademicHours += 72; // У аспирантов много исследовательских часов
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "PHD_RESEARCH", $"Исследование по теме: {DissertationTopic}", DateTime.Now);
    }

    // Реализация методов IResearcher
    void IResearcher.AddPublication(string title)
    {
        _publicationsCount++;
        _publications.Add(title);
        
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "PHD_PUBLICATION_ADDED", $"Добавлена публикация: {title}", DateTime.Now);
        _notificationService.SendNotification($"Ваша статья '{title}' опубликована", StudentId);
        
        Console.WriteLine($"{Name} опубликовал статью: '{title}'");
    }

    int IResearcher.GetPublicationCount()
    {
        return _publicationsCount;
    }

    List<string> IResearcher.GetPublications()
    {
        return new List<string>(_publications);
    }

    // Реализация методов ITeacherAssistant
    void ITeacherAssistant.ConductSeminar(string subject)
    {
        if (IsTeaching)
        {
            _taughtSeminars++;
            TeachingHours += 2;
            Console.WriteLine($"{Name} проводит семинар по предмету: {subject}");
            UpdateActivity();
            _eventLogger.LogEvent(StudentId, "SEMINAR_CONDUCTED", $"Проведен семинар по предмету: {subject}", DateTime.Now);
            _notificationService.SendNotification($"Вы провели семинар по предмету: {subject}", StudentId);
        }
        else
        {
            Console.WriteLine($"{Name} не имеет права проводить семинары");
        }
    }

    void ITeacherAssistant.GradeAssignments(int count)
    {
        if (IsTeaching)
        {
            Console.WriteLine($"{Name} проверяет задания у {count} студентов");
            TeachingHours += (int)(count * 0.5); // Явное приведение типа
            UpdateActivity();
            _eventLogger.LogEvent(StudentId, "ASSIGNMENTS_GRADED", $"Проверено заданий: {count}", DateTime.Now);
        }
        else
        {
            Console.WriteLine($"{Name} не имеет права проверять задания");
        }
    }

    public void PrepareForDefense()
    {
        int daysLeft = (DefenseDate - DateTime.Now).Days;
        Console.WriteLine($"{Name} готовится к защите диссертации. Осталось {daysLeft} дней");
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "DEFENSE_PREPARATION", $"Подготовка к защите, дней осталось: {daysLeft}", DateTime.Now);
    }

    public void SubmitDissertationDraft()
    {
        Console.WriteLine($"{Name} представил черновик диссертации по теме: '{DissertationTopic}'");
        UpdateActivity();
        _eventLogger.LogEvent(StudentId, "DISSERTATION_SUBMITTED", $"Представлен черновик диссертации: {DissertationTopic}", DateTime.Now);
        _notificationService.SendNotification($"Ваш черновик диссертации представлен", StudentId);
    }

    public override void AdvanceToNextCourse()
    {
        base.AdvanceToNextCourse();
        _eventLogger.LogEvent(StudentId, "PHD_ADVANCE", $"Аспирант переведен на курс {Course}", DateTime.Now);
        
        if ((DefenseDate - DateTime.Now).Days < 365)
        {
            Console.WriteLine($"Внимание! До защиты диссертации осталось меньше года!");
        }
    }
}

// Generic класс для учебных отделов
public class AcademicDepartment<T> where T : Student
{
    private List<T> students = new List<T>();
    private readonly INotificationService _notificationService;
    private readonly IEventLogger _eventLogger;
    public string DepartmentName { get; private set; }
    public string HeadOfDepartment { get; private set; }

    public AcademicDepartment(string departmentName, string headOfDepartment,
                             INotificationService notificationService, IEventLogger eventLogger)
    {
        DepartmentName = departmentName;
        HeadOfDepartment = headOfDepartment;
        _notificationService = notificationService;
        _eventLogger = eventLogger;
    }

    public void EnrollStudent(T student)
    {
        students.Add(student);
        _eventLogger.LogEvent(student.StudentId, "STUDENT_ENROLLED_IN_DEPARTMENT", 
                             $"Студент зачислен в отдел: {DepartmentName}, заведующий: {HeadOfDepartment}", DateTime.Now);
        _notificationService.SendNotification($"Вы зачислены в учебный отдел: {DepartmentName}", student.StudentId);
        Console.WriteLine($"Студент {student.Name} зачислен в отдел: {DepartmentName}");
    }

    public T FindStudent(string studentId)
    {
        return students.Find(s => s.StudentId == studentId);
    }

    public List<T> GetStudentsByCourse(int course)
    {
        return students.FindAll(s => s.Course == course);
    }

    public void ShowAllStudents()
    {
        Console.WriteLine($"\nУчебный отдел: {DepartmentName}, Заведующий: {HeadOfDepartment}");
        Console.WriteLine($"Количество студентов: {students.Count}");
        Console.WriteLine("Студенты:");
        foreach (var student in students)
        {
            student.DisplayInfo();
        }
    }

    public void AdvanceAllStudents()
    {
        foreach (var student in students)
        {
            student.AdvanceToNextCourse();
        }
    }

    public void SuspendAll()
    {
        foreach (var student in students)
        {
            student.Suspend();
        }
    }

    public void ActivateAll()
    {
        foreach (var student in students)
        {
            student.Activate();
        }
    }
}

// Generic класс для научных групп
public class ResearchGroup<T> where T : IResearcher
{
    private List<T> researchers = new List<T>();
    public string GroupName { get; private set; }
    public string ResearchArea { get; private set; }

    public ResearchGroup(string groupName, string researchArea)
    {
        GroupName = groupName;
        ResearchArea = researchArea;
    }

    public void AddResearcher(T researcher)
    {
        researchers.Add(researcher);
        Console.WriteLine($"Исследователь добавлен в научную группу {GroupName}");
    }

    public void ShowGroupInfo()
    {
        Console.WriteLine($"\nНаучная группа: {GroupName}, Область исследований: {ResearchArea}");
        Console.WriteLine($"Количество исследователей: {researchers.Count}");
        Console.WriteLine("Исследователи:");
        foreach (var researcher in researchers)
        {
            Console.WriteLine($"  - Публикаций: {researcher.GetPublicationCount()}");
        }
    }

    public int GetTotalPublications()
    {
        int total = 0;
        foreach (var researcher in researchers)
        {
            total += researcher.GetPublicationCount();
        }
        return total;
    }
}

// Класс Университет
public class University
{
    private List<Student> students = new List<Student>();
    private readonly INotificationService _notificationService;
    private readonly IEventLogger _eventLogger;

    public University(INotificationService notificationService, IEventLogger eventLogger)
    {
        _notificationService = notificationService;
        _eventLogger = eventLogger;
    }

    public void EnrollStudent(Student student)
    {
        students.Add(student);
        _eventLogger.LogEvent(student.StudentId, "STUDENT_ENROLLED", $"Студент зачислен в университет", DateTime.Now);
        Console.WriteLine($"Зачислен студент: {student.Name}");
    }

    public void OrganizeConsultation(string studentName1, string studentName2, string topic)
    {
        var student1 = students.Find(s => s.Name == studentName1);
        var student2 = students.Find(s => s.Name == studentName2);

        if (student1 != null && student2 != null)
        {
            Console.WriteLine($"Консультация: {studentName1} консультируется с {studentName2}");
            Console.WriteLine($"Тема: {topic}");
            
            _eventLogger.LogEvent(student1.StudentId, "CONSULTATION", $"Консультация с {studentName2} по теме: {topic}", DateTime.Now);
            _eventLogger.LogEvent(student2.StudentId, "CONSULTATION", $"Консультация с {studentName1} по теме: {topic}", DateTime.Now);
            
            student1.Study();
            student2.TakeExam();
        }
        else
        {
            Console.WriteLine("Ошибка: студенты не найдены.");
        }
    }

    public void ShowAllStudents()
    {
        Console.WriteLine("\nВсе студенты университета:");
        foreach (var student in students)
        {
            student.DisplayInfo(true);
        }
    }

    public void AdvanceAllStudents()
    {
        Console.WriteLine("Перевод всех студентов на следующий курс:");
        foreach (var student in students)
        {
            student.AdvanceToNextCourse();
        }
    }

    public void SuspendStudent(string studentId)
    {
        var student = students.Find(s => s.StudentId == studentId);
        student?.Suspend();
    }

    public void ActivateStudent(string studentId)
    {
        var student = students.Find(s => s.StudentId == studentId);
        student?.Activate();
    }
}

// Главная программа
var notificationService = new EmailNotificationService();
var eventLogger = new ConsoleEventLogger();

var university = new University(notificationService, eventLogger);

// Создание студентов с зависимостями
Student student1 = new Student("Иван Иванов", 19, "ivan@edu.ru", "+79002223344", "ул. Ленина, 10", "1234567890",
                              "STD-001", 2, 4.2m, true, notificationService, eventLogger);

BachelorStudent bachelor1 = new BachelorStudent("Анна Смирнова", 20, "anna@edu.ru", "+79003334455", "пр. Мира, 25", "0987654321",
                                               "BAC-202", 3, 4.5m, true, "Информатика", "ФИТ", false, "Очная",
                                               notificationService, eventLogger);

MasterStudent master1 = new MasterStudent("Петр Петров", 23, "petr@edu.ru", "+79004445566", "ул. Пушкина, 5", "1122334455",
                                         "MAG-777", 1, 4.8m, true, "Проф. Сидоров", "Машинное обучение", false,
                                         notificationService, eventLogger);

PhDStudent phd1 = new PhDStudent("Мария Козлова", 26, "maria@edu.ru", "+79005556677", "ул. Гагарина, 15", "5566778899",
                                "PHD-333", 4, 5.0m, true, "ИИ в образовании", "Кафедра ИВТ", 
                                new DateTime(2024, 12, 20), true, "Исследовательская группа",
                                notificationService, eventLogger);

// Зачисление в университет
university.EnrollStudent(student1);
university.EnrollStudent(bachelor1);
university.EnrollStudent(master1);
university.EnrollStudent(phd1);

Console.WriteLine("\n=== Журналирование событий и уведомления ===");
student1.ChangeContactInfo("newivan@edu.ru", "+79111111111");
student1.DisplayInfo(true);

Console.WriteLine("\n=== Учебный процесс ===");
student1.EnrollSubject("Математика");
IGradeable gradeableStudent = student1;
gradeableStudent.AddGrade("Математика", 4.5m);
student1.Study();
student1.TakeExam();

Console.WriteLine("\n=== Бакалавриат ===");
bachelor1.CompleteInternship();
Console.WriteLine($"Бакалавр готов к выпуску: {bachelor1.IsEligibleForGraduation()}");

Console.WriteLine("\n=== Магистратура ===");
IResearcher researcherMaster = master1;
researcherMaster.AddPublication("Новые методы анализа данных");
master1.ConsultWithAdvisor();
master1.TakeExam();

Console.WriteLine("\n=== Аспирантура ===");
IResearcher researcherPhD = phd1;
ITeacherAssistant teacherAssistant = phd1;

researcherPhD.AddPublication("ИИ в современном образовании");
teacherAssistant.ConductSeminar("Нейронные сети");
teacherAssistant.GradeAssignments(25);
phd1.PrepareForDefense();
phd1.SubmitDissertationDraft();

Console.WriteLine("\n=== Работа с интерфейсами ===");
Console.WriteLine($"Публикаций магистранта: {researcherMaster.GetPublicationCount()}");
Console.WriteLine($"Публикаций аспиранта: {researcherPhD.GetPublicationCount()}");
Console.WriteLine("Проведение семинара через интерфейс:");
teacherAssistant.ConductSeminar("Базы данных");

Console.WriteLine("\n=== Generic классы для учебных отделов ===");
AcademicDepartment<Student> generalDepartment = new AcademicDepartment<Student>("Общий учебный отдел", "Проф. Иванов",
                                                                               notificationService, eventLogger);
AcademicDepartment<BachelorStudent> bachelorDepartment = new AcademicDepartment<BachelorStudent>("Отдел бакалавриата", "Доц. Петрова",
                                                                                               notificationService, eventLogger);
AcademicDepartment<MasterStudent> masterDepartment = new AcademicDepartment<MasterStudent>("Отдел магистратуры", "Проф. Сидоров",
                                                                                         notificationService, eventLogger);

generalDepartment.EnrollStudent(student1);
generalDepartment.EnrollStudent(bachelor1);
generalDepartment.EnrollStudent(master1);
generalDepartment.EnrollStudent(phd1);

bachelorDepartment.EnrollStudent(bachelor1);
masterDepartment.EnrollStudent(master1);

Console.WriteLine("\n=== Научные исследовательские группы ===");
ResearchGroup<IResearcher> researchGroup = new ResearchGroup<IResearcher>("Исследовательская группа ИИ", "Искусственный интеллект");
researchGroup.AddResearcher(master1);
researchGroup.AddResearcher(phd1);
researchGroup.ShowGroupInfo();
Console.WriteLine($"Всего публикаций в группе: {researchGroup.GetTotalPublications()}");

Console.WriteLine("\n=== Консультации в университете ===");
university.OrganizeConsultation("Анна Смирнова", "Петр Петров", "Базы данных");

Console.WriteLine("\n=== Перевод на следующий курс ===");
university.AdvanceAllStudents();

Console.WriteLine("\n=== Все студенты университета ===");
university.ShowAllStudents();

Console.WriteLine("\n=== Управление статусами студентов ===");
university.SuspendStudent("STD-001");
university.ActivateStudent("STD-001");

Console.WriteLine("\n=== Расчет стипендий ===");
Console.WriteLine($"Стипендия студента1: {student1.CalculateScholarship():C}");
Console.WriteLine($"Стипендия бакалавра: {bachelor1.CalculateScholarship():C}");

Console.WriteLine("\n=== Информация из учебных отделов ===");
generalDepartment.ShowAllStudents();
bachelorDepartment.ShowAllStudents();
masterDepartment.ShowAllStudents();

Журнал: Студент NEW_PERSON, Событие: PERSON_CREATED, Детали: Создана персона: Иван Иванов, возраст: 19, Время: 22:55:48
Журнал: Студент STD-001, Событие: STUDENT_CREATED, Детали: Создан студент: Иван Иванов, курс: 2, средний балл: 4.2, Время: 22:55:48
Журнал: Студент NEW_PERSON, Событие: PERSON_CREATED, Детали: Создана персона: Анна Смирнова, возраст: 20, Время: 22:55:48
Журнал: Студент BAC-202, Событие: STUDENT_CREATED, Детали: Создан студент: Анна Смирнова, курс: 3, средний балл: 4.5, Время: 22:55:48
Журнал: Студент BAC-202, Событие: BACHELOR_CREATED, Детали: Создан бакалавр: Анна Смирнова, специальность: Информатика, факультет: ФИТ, Время: 22:55:48
Журнал: Студент NEW_PERSON, Событие: PERSON_CREATED, Детали: Создана персона: Петр Петров, возраст: 23, Время: 22:55:48
Журнал: Студент MAG-777, Событие: STUDENT_CREATED, Детали: Создан студент: Петр Петров, курс: 1, средний балл: 4.8, Время: 22:55:48
Журнал: Студент MAG-777, Событие: MASTER_CREATED, Детали: Создан магистрант: Петр Петров